**SHAPEFILE**

> The shapefile format is a geospatial vector data format for geographic information system (GIS) software. It is developed and regulated by Esri as a mostly open specification for data interoperability among Esri and other GIS software products. The shapefile format can spatially describe vector features: points, lines, and polygons, representing, for example, water wells, rivers, and lakes. Each item usually has attributes that describe it, such as name or temperature.
>
> Shapefile always consists of 5 files.

**GeoJSON**


> GeoJSON is a format for encoding a variety of geographic data structures.
>
>```
{
  "type": "Feature",
  "geometry": {
    "type": "Point",
    "coordinates": [125.6, 10.1]
  },
  "properties": {
    "name": "Dinagat Islands"
  }
}
```
>
> GeoJSON supports the following geometry types: Point, LineString, Polygon, MultiPoint, MultiLineString, and MultiPolygon. Geometric objects with additional properties are Feature objects. Sets of features are contained by FeatureCollection objects.



**PROJECTIONS**


> In cartography, map projection is the term used to describe a broad set of transformations employed to represent the two-dimensional curved surface of a globe on a plane. In a map projection, coordinates, often expressed as latitude and longitude, of locations from the surface of the globe are transformed to coordinates on a plane. Projection is a necessary step in creating a two-dimensional map and is one of the essential elements of cartography.
>
> All projections of a sphere on a plane necessarily distort the surface in some way and to some extent.
>
> Read more here: https://gisgeography.com/map-projections/
>
> And here you can look at projection types: https://gisgeography.com/map-projection-types/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install geopandas
!pip install geojson

# Convert geojson into shapefile

In [ ]:
geoj = {"type": "FeatureCollection","name": "Hyde_Park","crs": { "type": "name", "properties": { "name": "urn:ogc:def:crs:OGC:1.3:CRS84" } },"features": [{ "type": "Feature", "properties": { "id": 3 }, "geometry": { "type": "Polygon", "coordinates": [ [ [ -0.18748934798873, 51.502397801761354 ], [ -0.184661560785939, 51.501970555509452 ], [ -0.182986851860014, 51.501594575494373 ], [ -0.182163224519395, 51.501526215158371 ], [ -0.180296335880659, 51.50156039533919 ], [ -0.178539264220673, 51.501662935727843 ], [ -0.177001826518185, 51.501645845679086 ], [ -0.174228947804768, 51.501850925841126 ], [ -0.165992674398581, 51.502329442630227 ], [ -0.158662391067074, 51.502807954394761 ], [ -0.153720627023361, 51.503218103336671 ], [ -0.152787182703994, 51.503594069957224 ], [ -0.15232046054431, 51.504363082928876 ], [ -0.152018463852749, 51.505490945145304 ], [ -0.152155735076186, 51.505969423712344 ], [ -0.152375369033684, 51.506379544199341 ], [ -0.154434437385231, 51.508002901568808 ], [ -0.156520959981465, 51.509574939894684 ], [ -0.157426950056146, 51.510309678337563 ], [ -0.157976034949892, 51.511027318400807 ], [ -0.158827116535198, 51.512479650606025 ], [ -0.160996001865494, 51.512325876446553 ], [ -0.161874537695487, 51.513026398969167 ], [ -0.172224787942596, 51.511967068055817 ], [ -0.175244754858198, 51.511659515757266 ], [ -0.180213973146598, 51.510924799083888 ], [ -0.182273041498144, 51.510617239748903 ], [ -0.182931943370639, 51.51051471950916 ], [ -0.185128282945623, 51.510378025497296 ], [ -0.187763890435603, 51.510138809989648 ], [ -0.19114076253214, 51.510002114849897 ], [ -0.189136602669967, 51.506037777383185 ], [ -0.189054239935905, 51.505542210946388 ], [ -0.18748934798873, 51.502397801761354 ] ] ] } }]}

In [ ]:
#convert geojson file to shapefile and transfer the coordinate system
import geopandas
import geojson 
import subprocess

def geojson_to_shapefile(geoj):
    """
    This function converts geojson file to shapefile and transfers the coordinate system (need only once)
    """
    #input the geojson file
    data = geoj
    with open('data.geojson', 'w') as f:
        geojson.dump(data, f)

    args = ['ogr2ogr', '-f', 'ESRI Shapefile', '/content/drive/MyDrive/GIS_tutorials/2_python_main_gis_functions/temp/destination_data.shp', 'data.geojson']
    subprocess.Popen(args)

    ## convert data from 4326 to 3857
    data = geopandas.read_file("/content/drive/MyDrive/GIS_tutorials/2_python_main_gis_functions/temp/destination_data.shp")
    # change CRS to epsg 3857
    data = data.to_crs(epsg=3857)
    # write shp file
    data.to_file("/content/drive/MyDrive/GIS_tutorials/2_python_main_gis_functions/my_shapefile/myshapefile.shp")

In [ ]:
geojson_to_shapefile(geoj)

# Additional exercise: write and read shapefile

In [ ]:
### Lets start by reading a csv of coordinates

directory= '/content/drive/MyDrive/GIS_tutorials/2_python_main_gis_functions/data_package_beg/'
fileName = directory + "Points_training.csv"

### reading file line by line

f = open(fileName, 'r') 
for line in f: 
    print (line)
f.close()

Now we can start using the ogr and gdal libraries. 
<br></br>
writing a point shapefile

In [ ]:
#https://pcjericks.github.io/py-gdalogr-cookbook/geometry.html#create-a-point
from osgeo import ogr, osr

#fileName = r"D:\Cardiff\day1\data_package_beg\Points_training.csv"

### reading file line by line
lsData = []
f = open(fileName, 'r') 
for line in f: 
    data = line.split(',') ### we need to split the data based on comma!
    lsData.append(data)
f.close()

### So we have our data!! now create the shapefile

# 1. set up the shapefile driver
driver = ogr.GetDriverByName("ESRI Shapefile")

# 2.create the file
data_source = driver.CreateDataSource(directory + "points.shp")

# 3. set the spatial reference in this case UTM 30N
srs = osr.SpatialReference()
srs.ImportFromEPSG(32630) ### think about this a variable - build a function **

# 4. create the layer
layer = data_source.CreateLayer("points", srs, ogr.wkbPoint) ## look at ogr cook book

# 5. Add the fields to the shapefile
layer.CreateField(ogr.FieldDefn("Y", ogr.OFTReal)) ## data types
layer.CreateField(ogr.FieldDefn("X", ogr.OFTReal))

# 6. set the geometry as a POINT!
point = ogr.Geometry(ogr.wkbPoint)
# loop through the data
for i in range(1,len(lsData)):
    #create the feature
    feature = ogr.Feature(layer.GetLayerDefn())
    feature.SetField("Y", lsData[i][1]) ## i is iterator, 1 is position.. storing y,x not x,y
    feature.SetField("X", lsData[i][0])
    # add the point
    point.AddPoint(float(lsData[i][0]), float(lsData[i][1]), i)
    # set the wkt (well known text)
    wkt = point.ExportToWkt() ## print this if not sure..
    print (wkt)
    # Create the point from the Well Known Txt
    point = ogr.CreateGeometryFromWkt(wkt)

    # Set the feature geometry using the point
    feature.SetGeometry(point)
    # Create the feature in the layer (shapefile)
    layer.CreateFeature(feature)
    # Destroy the feature to free resources
    feature.Destroy()

# 7. Destroy the data source to free resources
data_source.Destroy() ## destroy can be a scary term, it would be better if it was called something else but... nothing we can do (well open source?)

In [ ]:
### Reading a shapefile and printing its spatial reference
# Import the necessary modules
from  osgeo import ogr, osr

driver = ogr.GetDriverByName('ESRI Shapefile')
shp = driver.Open(directory + 'points.shp')

# Get Projection from layer
layer = shp.GetLayer()
spatialRef = layer.GetSpatialRef()
print (spatialRef)
print ("")
### get the number of features in a shapefile
featureCount = layer.GetFeatureCount()
print ("feature count:", featureCount)
print ("")
## get all the values in X field
print ("printing all the x values")
for feature in layer:
    print (feature.GetField("X"))